# Study Selection
partial version, contains full-text assessment 

In [1]:
import pandas as pd
import os
import json
import logging
import time

from utils.General.quicker import Quicker, QuickerData, QuickerStage
from utils.Evidence_Assessment.paper import Paper
from utils.Evidence_Assessment.outcome import Outcome
from utils.Evidence_Assessment.evidence import Evidence
from utils.logging import setup_logging

setup_logging()

# Hyperparameters
YOUR_CONFIG_PATH = 'YOUR_CONFIG_PATH' # e.g. 'config/config.json'
YOUR_DATASET_PATH = 'YOUR_DATASET_PATH' # e.g. 'data/2021ACR RA'
YOUR_QUESTION_DECOMPOSTITION_PATH =  'YOUR_QUESTION_DECOMPOSTITION_PATH' # e.g. 'data/2021ACR RA/Question_Decomposition'
YOUR_LITERATURE_SEARCH_PATH = 'YOUR_LITERATURE_SEARCH_PATH' # e.g. 'data/2021ACR RA/Literature_Search'
YOUR_STUDY_SELECTION_PATH = 'YOUR_STUDY_SELECTION_PATH' # e.g. 'data/2021ACR RA/Study_Selection'
YOUR_EVIDENCE_ASSESSMENT_PATH = 'YOUR_EVIDENCE_ASSESSMENT_PATH' # e.g. 'data/2021ACR RA/Evidence_Assessment'
YOUR_PAPER_LIBRARY_PATH = 'YOUR_PAPER_LIBRARY_PATH' # e.g. 'data/2021ACR RA/Paper_Library'
record_included_path = "your_record_included_path" # json file. You can use the file saved by Quicker.quicker_data.to_json method.

disease = 'disease_or_topic_of_your_question' # Disease name or clinical topic of your clinical question. e.g. 'Rheumatoid Arthritis (RA)'
pico_idx = "xxxxxxxxx" # PICO index of the clinical question, saved in your PICO_Information.json file


study_design_list = ['randomized clinical trial'] # Study design list for inclusion criteria

In [ ]:
# The following example code is used to load included records from the json file by Quicker.quicker_data.to_json method.
with open(record_included_path, 'r') as f:
    record_included_dict = json.load(f)
record_included_list = record_included_dict['record_included_studies'] 

In [4]:
quicker_data = QuickerData(disease=disease,pico_idx=pico_idx)
quicker = Quicker(
    config_path=YOUR_CONFIG_PATH,
    question_deconstruction_database_path=YOUR_QUESTION_DECOMPOSTITION_PATH,
    literature_search_database_path=YOUR_LITERATURE_SEARCH_PATH,
    study_selection_database_path=YOUR_STUDY_SELECTION_PATH,
    evidence_assessment_database_path=YOUR_EVIDENCE_ASSESSMENT_PATH,
    quicker_data=quicker_data,
    paper_library_base=YOUR_PAPER_LIBRARY_PATH,
)

quicker.comparator_postfix_map = {} #! set {} if only one comparator


question_deconstruction_datapath = os.path.join(
    quicker.question_deconstruction_database_path, 'PICO_Information.json'
)
question_deconstruction_data = pd.read_json(
    question_deconstruction_datapath, dtype={'Index': str}
)
question_deconstruction_data = question_deconstruction_data[
    question_deconstruction_data['Index'] == pico_idx
]
original_qd_dict = question_deconstruction_data.to_dict(orient='records')

data_dict = dict(
    clinical_question=original_qd_dict[0]['Question'],
    population=original_qd_dict[0]['P'],
    intervention=original_qd_dict[0]['I'],
    comparison=original_qd_dict[0]['C'],
    study=study_design_list,
    outcome=original_qd_dict[0]['O'],

)

quicker._add_data_to_quickerdata_for_test(
    stage=QuickerStage.LITERATURE_SEARCH,
    default_value=data_dict,
)

quicker.set_inclusion_exclusion_criteria(inclusion_criteria='', exclusion_criteria='')

In [6]:
print(quicker)
# print(quicker.quicker_data)
print(quicker.quicker_data.check_stage_state())

Quicker: STUDY_SELECTION
StageState.NOT_STARTED


In [ ]:
#! Please implement the get_pdf method of class Paper in utils\Evidence_Assessment\paper.py
_, _, _ = quicker.select_studies_by_full_text_assessment(record_included_list)